# Modelling a perfectly spherical star in python


# Libraries

In [1]:
import math
import decimal #The sun is roughly ~10^27 m^3, making it safe to say the volumes of gases we'll be dealing with will be around ~10^24 m^3, which is far beyond what python's float data type can handle.
import re
import random
from operator import truediv
import hashlib
import numpy as np
import pandas as pd #< To work with mostIsotopesMolarMasses.csv
import pygame
import sys
import logging
logging.basicConfig(level=logging.INFO)
from debugpy.launcher.winapi import kernel32

pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Useful constants
Makes life easier.

In [2]:
N_a = 6.02214076 * 10 ** (23)
mu = 1.66054*10**(-27)
e: float = math.e
G: float = 6.6743*10**(-11)
pi: float = math.pi
c: float = 299792458
h: float = 6.6260*10**(-34)
sigma: float = 5.6703*10**(-8)
k: float = 1.38 * 10**(-23)
R: float = 8.3145
epsilon_0: float = 8.854*10**(-12)
e_charge: float = 1.602*10**(-19)
r_0: float = 1.4*10**(-15)

# Simulation time interval

The collapse of gas clouds into stars typically takes millions of years. Unfortunately my computer isn't a supercomputer, and we don't have all year to wait for the results, so we have to take some shortcuts.

We can take much larger time intervals at the beginning of the simulation, where changes in the system are much smaller, and errors will accumulate less. Of course, different sizes and masses of gas clouds will take different times to collapse, so using the free fall equation would tell us what kind of time scale we are looking at.

$$t_{ff} = \sqrt{\frac{3\pi}{32G \rho}}$$

In [3]:
SIMULATION_TIME_INTERVAL = 1

# Getting the information out of reactionsInfo.txt

reactionsInfo.txt holds the relevant data for what Reaclib1 file holds the data for which reaction, as well as what the reactants and products are.

The output at the bottom line should look like: 71c7209dc0db7b35cb1201198606aed2

If so, most likely there is a problem in reactionsInfo.txt.

In [4]:
f = open("reactionsInfo.txt", "r")

df = pd.read_csv("C:/Users/dsyao/PycharmProjects/compPhysStars/models/molar_masses/mostIsotopesMolarMasses.csv")

reactionTable = f.read().splitlines()
reactionTable.pop(0)
logging.info(reactionTable)
logging.info("----------------------------------------------------------------------")
reactionTable = [row.split(",") for row in reactionTable]
logging.info(reactionTable)
logging.info("----------------------------------------------------------------------")
reactionTable = [[[row[0]]] + [row[1].split("/")] + [row[2].split("/")] for row in reactionTable]
logging.info(reactionTable)
logging.info(reactionTable[0])
f.close()
hashString = f"{reactionTable}"
hexdigest = hashlib.md5(hashString.encode("utf-8")).hexdigest()
logging.info(hexdigest)
if hexdigest != "71c7209dc0db7b35cb1201198606aed2":
    logging.warning("Erroneous reaction hash")

INFO:root:['ar36-ag-ca40-ths8,1-4-2/1-36-18,1-40-20,', 'b8-a-he4-wc12,1-8-5,1-4-2,', 'be7--li7-ec,1-7-4,1-7-3,', 'be7-pg-b8-nacr,1-7-4/1-1-1,1-8-5,', 'be9-an-c12-cf88,1-9-4/1-4-2,1-12-6/1-1-0,', 'c12-ag-o16-nac2,1-12-6/1-4-2,1-16-8,', 'c12-pg-n13-ls09,1-12-6/1-1-1,1-13-7,', 'c13-pg-n14-nacr,1-13-6/1-1-1,1-14-7,', 'ca40-ag-ti44-chw0,1-40-20/1-4-2,1-44-22,', 'd-pg-he3-de04,1-2-1/1-1-1,1-3-2,', 'f17--o17-wc12,1-17-9,1-17-8,', 'f18--o18-wc12,1-18-9,1-18-8,', 'f19-pa-o16-nacr,1-19-9,1-16-8/1-1-1,', 'he3-ag-be7-cd08,1-3-2/1-4-2,1-7-4,', 'he3-he3pp-he4-nacr,1-3-2/1-3-2,1-4-2/2-1-1,', 'he3-p-he4-bet,1-3-2/1-1-1,1-4-2,', 'he4-aag-c12-fy05,1-4-2,1-12-6,', 'he4-nag-be9-ac12,1-4-2/1-9-4,1-12-6,', 'li7-pa-he4-de04,1-7-3,1-4-2/1-3-1,', 'mg24-ag-si28-st08,1-24-12/1-4-2,1-28-14,', 'n13--c13-wc12,1-13-7,1-13-6/,', 'n14-pg-o15-im05,1-14-7/1-1-1,1-15-8,', 'n15-pa-c12-nacr,1-15-7,1-12-6/1-1-1,', 'n15-pg-o16-li10,1-15-7/1-1-1,1-16-8,', 'ne20-ag-mg24-il10,1-20-10/1-4-2,1-24-12,', 'o15--n15-wc12,1-15-8,1-15-

# Reading Reaclib1 files

In [5]:
#This code was not written by me. It was written by Deepseek

def parse_reaclib1_file(filename):
    """
    Parses a Reaclib1 file and extracts the reaction details and coefficients.
    """
    reactions = []
    with open(filename, 'r') as file:
        lines = file.readlines()

        # Skip the header (chapter number)
        i = 1  # Start reading from the second line

        while i < len(lines):
            # First line of the set entry
            line1 = lines[i].strip()
            if not line1:  # Skip empty lines
                i += 1
                continue

            # Extract reactants and products
            reactants = [line1[5:10].strip(), line1[10:15].strip(), line1[15:20].strip()]
            products = [line1[20:25].strip(), line1[25:30].strip(), line1[30:35].strip()]

            # Extract set label, rate type, reverse flag, and Q value
            set_label = line1[43:47].strip() if len(line1) >= 47 else ''
            rate_type = line1[47] if len(line1) >= 48 else ''
            reverse_flag = line1[48] if len(line1) >= 49 else ''
            q_value = float(line1[52:64]) if len(line1) >= 64 else 0.0

            # Second line of the set entry (first four coefficients)
            line2 = lines[i + 1].strip() if i + 1 < len(lines) else ''
            # Split line2 into coefficients using 'e+' or 'e-' as delimiters
            coeffs_line2 = []
            if line2:
                # Use a regular expression to split the line into valid scientific notation strings
                import re
                parts = re.findall(r'[-+]?\d*\.\d+[eE][-+]?\d+', line2)
                coeffs_line2 = [float(part) for part in parts]

            # Third line of the set entry (last three coefficients)
            line3 = lines[i + 2].strip() if i + 2 < len(lines) else ''
            # Split line3 into coefficients using 'e+' or 'e-' as delimiters
            coeffs_line3 = []
            if line3:
                # Use a regular expression to split the line into valid scientific notation strings
                import re
                parts = re.findall(r'[-+]?\d*\.\d+[eE][-+]?\d+', line3)
                coeffs_line3 = [float(part) for part in parts]

            # Combine all coefficients
            coefficients = coeffs_line2 + coeffs_line3
            if len(coefficients) < 7:
                coefficients.extend([0.0] * (7 - len(coefficients)))  # Pad with zeros if necessary

            # Store the reaction details and coefficients
            reaction = {
                'reactants': reactants,
                'products': products,
                'set_label': set_label,
                'rate_type': rate_type,
                'reverse_flag': reverse_flag,
                'q_value': q_value,
                'coefficients': coefficients
            }
            reactions.append(reaction)

            # Move to the next set entry
            i += 3

    return reactions


def calculate_reaction_rate(T9, coefficients):
    """
    Calculates the reaction rate at a given temperature T9 (in 10^9 K).
    """
    a0, a1, a2, a3, a4, a5, a6 = coefficients
    log_rate = a0 + a1 / T9 + a2 / T9 ** (1 / 3) + a3 * T9 ** (1 / 3) + a4 * T9 + a5 * T9 ** (5 / 3) + a6 * np.log(T9)
    return np.exp(log_rate)

In [6]:
reactionsDetailList = []
for i in reactionTable:
    e = str(i[0][0])

    reactions = parse_reaclib1_file("models/rates/" + e)

    coefficientsList = []
    for reaction in reactions:
        coefficientsList.append(reaction['coefficients'])

    logging.info(coefficientsList)


    i.append(coefficientsList)
logging.info("")
for i in reactionTable:
    logging.info(f"{i}\n")

INFO:root:[[52.3486, 0.0, -71.0046, 4.0656, -5.26509, 0.683546, -0.666667]]
INFO:root:[[-0.105148, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
INFO:root:[[-23.8328, 0.0, 0.0, 3.02033, -0.0742132, -0.00792386, -0.650113]]
INFO:root:[[7.73399, -7.345, 0.0, 0.0, 0.0, 0.0, -1.5], [12.5315, 0.0, -10.264, -0.203472, 0.121083, -0.00700063, -0.666667]]
INFO:root:[[11.744, -4.179, 0.0, 0.0, 0.0, 0.0, -1.5], [-1.48281, -1.834, 0.0, 0.0, 0.0, 0.0, -1.5], [-9.51959, -1.184, 0.0, 0.0, 0.0, 0.0, -1.5], [31.464, 0.0, -23.87, 0.566698, 44.0957, -314.232, -0.666667], [19.2962, -12.732, 0.0, 0.0, 0.0, 0.0, 0.0]]
INFO:root:[[254.634, -1.84097, 103.411, -420.567, 64.0874, -12.4624, 137.303], [69.6526, -1.39254, 58.9128, -148.273, 9.08324, -0.541041, 70.3554]]
INFO:root:[[17.1482, 0.0, -13.692, -0.230881, 4.44362, -3.15898, -0.666667], [17.5428, -3.77849, -5.10735, -2.24111, 0.148883, 0.0, -1.5]]
INFO:root:[[13.9637, -5.78147, 0.0, -0.196703, 0.142126, -0.0238912, -1.5], [15.1825, -13.5543, 0.0, 0.0, 0.0, 0.0, -1.5], [

# Defining elements

In [7]:
class element:
    #Proton number
    Z: int
    #Nucleon number
    N: int

    def __eq__(self, other):
        return isinstance(other, element) and self.Z == other.Z and self.N == other.N

    def __hash__(self):
        return hash((self.Z, self.N))

    def __init__(self, Z: int, N: int):
        self.Z = Z
        self.N = N


# Maxwell-Boltzmann cumulative distribution
This will be used to determine what proportion of particles near the surface have enough KE to overcome GPE near the surface. I will then cut down those numbers by first cutting it in half to model the particles heading back into the star. I will later attempt to make a formula linking density, gas temperature and particle speed to further find what percentage of particles will collide with the surrounding gas particles and not escape.

In order to do this, we need to find the integral of the Maxwell-Boltzmann distribution function, so that we can find the cumulative distribution function.

$$f(v)=(\frac{m}{2 \pi k_B T})^{3/2}4 \pi v^2 e^{-\frac{mv^2}{2 k_B T}}$$

$$\int f(v)dv=\int (\frac{m}{2 \pi k_B T})^{3/2}4 \pi v^2 e^{-\frac{mv^2}{2 k_B T}} dv$$

$$P(v_0, v_1, m)=4 \pi (\frac{m}{2 \pi k T})^{\frac{3}{2}}(-\frac{kT}{m}ve^{-\frac{mv^2}{2kT}}+\frac{\sqrt{2 \pi k^3 T^3}}{2m}\erf({\sqrt{\frac{m}{2kT}}v}))\Bigg|_{v_0}^{v_1} $$
This function is given as a function of velocity. Converting it to be a function of $KE$ would be too messy.

To make this easier to read, I'll break this down into a few functions. Along with this,any particle with energy higher than the minimum energy will be accepted, so $v_1=\infty$, and so the new euqation can be written:

$$P(v, m)=1-k_0(m)(k_1(v)k_2(m, v)+k_3(m)\erf({\sqrt{\frac{m}{2kT}}v})) $$
Where:
$$k_0(m) = 4 \pi (\frac{m}{2 \pi k T})^{\frac{3}{2}}$$
$$k_1(v) = -\frac{kT}{m}v$$
$$k_2(m, v) = e^{-\frac{mv^2}{2kT}}$$
$$k_3(m) = \frac{\sqrt{2 \pi k^3 T^3}}{2m}$$

This function does not have a concise analytical solution. We will have to take approximations to be able to do this.
We will use the math library to estimate the error function (erf).



In [8]:
def BoltzMannProbabliltyDistribution(E_min: float, mass: float, T: float) -> float: #Untested
    v_min = math.sqrt((2*E_min)/(mass))
    k0 = 4 * pi * math.sqrt(((mass)/(2 * pi * k * T)**2))
    k1 = (-k * T * v_min)/mass
    k2 = math.exp((-mass*v_min*v_min)/(2*k*T))
    k3 = (math.sqrt(2*pi*k*k*k*T*T*T))/(2*mass)
    erf = math.erf(math.sqrt((mass)/(2*k*T))*v_min)

    probability = 1-k0*(k1*k2+k3+erf)

    return probability

# The star model

The star will be modelled as a sphere, with many layers of differing compsosition. Each layer will consist of several element, at uniform average temperature. By the nuclear fusion probability function, the amount of that layer that fuses will be calculated, and the new materials will be mixed into the layer. Heavier elements will move towards the core. Descending layers will also take on a larger thickness as the radius grows smaller.

The mass of the star will be calculated by summing the masses of all the layers, as well as adding the mass from the kinetic energy of the molecules.

The star will lose mass via radiation and particles escaping via evaporation.

We can find the power output of the star by using the Stefan-Boltzmann law, assuming the star is a perfect black body.
$$P = \sigma A T^4$$

Evaporation can also be found by finding the proportion of particles at the surface which have sufficient energy to escape the gravitiational field of the sun. This is modelled by using the Maxwell-Boltzmann distribution to find the proportion of particles at the surface of the sun with sufficient energy to escape.

Stars form by clouds of gas collapsing in on themselves, and the gravitational potential energy being converted into kinetic energy. The high temperatures and pressures then drive fusion, which increases the temperature of the gases, increasing pressure and thereby pushing outwards and keeping the star in equilibrium. I will also assume that plasma acts as an ideal gas (even though it doesn't).

$$pV = nRT$$
$$kE = \frac{3}{2} kT$$

As layer n falls, its GPE is converted into KE, where r_0 is the radius between the center of the star and the bottom of the given layer and r_1, the top.

$$\Delta Ke = \frac{2GM_n \sum^{i}_{i=n-1}M_i}{r_{n0} + r_{n1}} - \frac{2GM_n \sum^{i}_{i=n-1}M_i}{r_{n0} + r_{n1} - \Delta r}$$

We can say that $\frac{r_{n0} + r_{n1}}{2} = r$, where R is the total radius of the star. So:

$$\Delta Ke = \frac{GM_n \sum^{i}_{i=n-1}M_i}{r} - \frac{2GM_n \sum^{i}_{i=n-1}M_i}{r - \Delta r}$$

## Forces on a Gas Layer

We model the changes in the layers of the star by measuring the change in the upper bound of each layer, then fixing that value to be the bottom layer of the layer above. The forces acting on any one layer are: gravity, pressure from within the layer, pressure from upper layer, radiation pressure from upper layers, radiation pressure from lower layers.

For layer $n$ or $N$

$$F_g = \frac{GM_n \sum^{i}_{i=n-1}M_i}{r^2}$$

However, there is also a resistive force from the pressure of the upper layer (including radiation pressure), where $\epsilon_j$ is the fraction of radiative pressure left over after passing through layer $j$.

$$F_p = P_{n+1}A + \sum^{N}_{i=n+1} \prod^{N}_{j=n+1} \epsilon_j P_{ri}A = 4 P_{n-1} \pi r_{n0}^2 + P_{r} r_{n0}^2$$

$$F_{total} = F_g - F_{pu} +F_{pl} = \frac{GM_n \sum^{n-1}_{i=1}M_i}{r^2} - 4 P_{n-1} \pi r_{n0}^2 + \sum^{n-1}_{i=1} \prod^{n-1}_{j=1} \epsilon_j P_{ri}A$$


## Cumulative error and motion

Even given our extremely simplified model, the equations describing the layers making up our star model would be much too complicated for me to do. Given the above euqation for force on a layer, the algorithm will calculate how far the layer will move, assuming the force is constant over the simulation time. This is assuming $\Delta r << R$, such that $F_g(t+\Delta t) \approx F_g(t)$ and that $F_p(t+\Delta t) \approx F_p(t)$. However, this method leads to errors, as in a real system, some forces would change as the system evolves, but we do not take that into account in our simulation.

## Adiabatic Heating

One of the main mechanisms by which gases heat up in a star is via adiabatic heating. This is the effect by which compressing a gas causes it to heat up. The equations defining it are:

$$\gamma = \frac{f + 2}{f}$$

Where because all the particles in our star are monatomic, $f = 3$

$$\gamma = \frac{5}{3}$$

And,

$$T_2 = T_1(\frac{P_2}{P_1})^{\frac{\gamma - 1}{\gamma}}$$
$$T_2 = T_1(\frac{P_2}{P_1})^{\frac{2}{5}}$$

## Fusion

The REACLIB database returns reaction rates in the form:

$$\lambda = N_a \langle\sigma v\rangle$$

And to use this, we will use the formulas:

For reaction $A + B$

$$r = n_a n_b \langle \sigma v \rangle$$

And for reaction $A + A$

$$r = 0.5 n_a^2 \langle \sigma v \rangle$$

And finally

$$\Delta M_i = r N_a m_i \Delta t$$

Where $m_i$ is the molar mass of $I$

## Heat Transfer

In stars, heat is transferred through it via convection, as well as radiative heating. However, for the first one, our model isn't exacly the best to try and model this, however it might be worth trying to simulate it using conduction between layers. The second major mode of heat transfer is radative heating, which will also cause radiation pressure.

# Order of calculations

1. Calculate forces on each layer
2. Calculate motion of each layer

    a. Calculate motion of upper bound for all layers

    b. Fix lower bound for all layers as upper bound of lower layer.

3. Calculate adiabatic heating

    a. Store previous pressure

    b. Calculate new pressure value

    c. Calculate new temperature
4. Calculate heat transfer between layers
5. Calculate separation of heavy elements between layers
6. Calculate energy radiated away, as well as the spectrum
7. Calculate change in temperature and composition by fusion

This is not perfect, since in reality all these would be happening at the same time. This will also probably lead to some violations of the first law of thermodynamics.

In [9]:
class starLayer:
    #                                                       \/ Mass in Kg     \/ Temp in °K      \/ Radius in m                    \/ Velocity in ms^-1
    def __init__(self, composition: list[tuple[element, decimal.Decimal]], temperature: float, radius_0: float, radius_1: float, velocity: float):
        self.composition = composition
        self.temperature = temperature #We're assuming temperature is homogenous within layers for simplicity's sake
        self.radius_0 = radius_0#We're also assuming density is even within layers
        self.radius_1 = radius_1
        self.velocity = velocity
        self.projectedVelocity = 0
 #       temp = 0
  #      for i in self.composition:
   #         temp += i[1]
        self.mass = np.sum([mass for _, mass in self.composition]) #needs to be recalculated at the beginning of every time interval
        self.pressure = 0 #needs to be recalculated at the beginning of every time interval
        self.elementSet = {(e.Z, e.N) for e, _ in self.composition} #needs to be recalculated at the beginning of every time interval
        self.volume = (4/3)*pi*(self.radius_1**3-self.radius_0**3) #needs to be recalculated at the beginning of every time interval
        self.numberOfMoles = 0 #needs to be recalculated at the beginning of every time interval

    #--STUFF THAT NEEDS TO BE RECALCULATED AT THE BEGINNING OF EVERY TIME INTERVAL -------

    def recalculateElementSet(self):
        self.elementSet = {(e.Z, e.N) for e, _ in self.composition}

    def calculateNumberOfMoles(self): #Untested
        self.numberOfMoles = sum(tup[1]/df.loc[(df["Z"] == tup[0].Z) & (df["N"] == tup[0].N), "MM"].values for tup in self.composition) * 1000

    def calculateTotalMass(self): #Untested
        self.mass = sum(tup[1] for tup in self.composition)

    def recalculateVolume(self):
        self.volume = (4/3)*pi*(self.radius_1**3-self.radius_0**3)

    def calculatePressure(self): #Tested, given test params: composition=[(element(Z = 8, N = 16), 0.016)], temperature=300, radius_0=0, radius_1=0.62035, velocity=0, output: 2495.14897033
        moles = self.numberOfMoles
        self.pressure = (moles * R * self.temperature)/self.volume
        return self.pressure

    #-----------------------------------------------------------------



    def fusion(self): #Untested
        random.shuffle(reactionTable) #This is to prevent the reactions to be calculated in the same order every time. If this happens, then it is likely that reactions near the start of the list are overly represented.
        for i in reactionTable:
            self.elementSet = {(e.Z, e.N) for e, _ in self.composition}
            if self.checkPresence(i) == True:
                #print("found!")

                print(i[0])
                rate = 0.0

                for z in i[3]:
                    # \/ Rate is in cm^3 mol^-1 s^-1, we want cm^3 s^-1
                    #ratesList = parse_reaclib1_file(f"C:/Users/dsyao/PycharmProjects/compPhysStars/models/rates/{i[0][0]}")

                    #for j in ratesList:
                    rate += calculate_reaction_rate(T9 = self.temperature/1000000000, coefficients=z)
                    logging.info(rate)

                print(f"volume: {self.volume}")
                print(f"rate: {rate}")

                rate = rate/(N_a * 1000000)

                if len(i[1]) == 2:
                    massDefct = 0.0
                    temp3 = i[1][0].split("-")
                    temp4 = i[1][1].split("-")
                    el1 = element(Z = int(temp3[2]), N = int(temp3[1]))
                    massDefct += df.loc[(df["Z"] == int(temp3[2])) & (df["N"] == int(temp3[1])), "MM"].values[0] * int(i[1][0][0])
                    mass1 = next((mass for e, mass in self.composition if el1.Z == el1.Z and e.N == el1.N), 0)
                    el2 = element(Z = int(temp4[2]), N = int(temp4[1]))
                    massDefct += df.loc[(df["Z"] == int(temp4[2])) & (df["N"] == int(temp4[1])), "MM"].values[0] * int(i[1][1][0])
                    mass2 = next((mass for e, mass in self.composition if el2.Z == el2.Z and e.N == el2.N), 0)
                    print(f"moles1: {self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1)}, moles2: {self.returnSpecificNumberOfMoles(elementt = el2, mass = mass2)}")
                    Y_1 = (self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1) * N_a)/self.volume
                    Y_2 = (self.returnSpecificNumberOfMoles(elementt = el2, mass = mass2) * N_a)/self.volume

                    print(f"Y_1: {Y_1}")
                    print(f"Y_2: {Y_2}")
                    #molCm3 = rate * ((self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1) * self.returnSpecificNumberOfMoles(elementt = el2, mass = mass2))/(self.volume ** 2))
                    molCm3 = (Y_1 * Y_2 * rate * self.volume)/N_a
                    logging.info(molCm3)
                    totalChangeMoles = (molCm3) * SIMULATION_TIME_INTERVAL# * self.volume
                    productEls = []
                    for w in i[2]:
                        _, N11, Z11 = w.split("-")
                        productEls.append((element(Z = int(Z11), N = int(N11)), w[0]))
                        massDefct -= df.loc[(df["Z"] == int(Z11)) & (df["N"] == int(N11)), "MM"].values[0] * int(w[0])
                    for m, (element1, mass) in enumerate(self.composition):
                        if element1 == el1:
                            self.composition[m] = (el1, self.composition[m][1] - totalChangeMoles * self.returnMolarMass(el1))
                        if element1 == el2:
                            self.composition[m] = (el2, self.composition[m][1] - totalChangeMoles * self.returnMolarMass(el2))
                        if element1 in productEls:
                            productEls.remove(element1)
                            self.composition[m] = (element1, self.composition[m][1] + totalChangeMoles * self.returnMolarMass(element1))
                    for q in productEls:

                        self.composition.append((q[0], totalChangeMoles * self.returnMolarMass(q[0]) * int(q[1])))
                    self.calculateNumberOfMoles()#This might be too much, and may slow down the operation too much to justify the small amount of extra accuracy. If true, remove it.
                    deltaE = massDefct * mu * c * c
                    self.temperature += (2 * deltaE)/(3 * self.numberOfMoles * R)
                    print("RAN!")
                else: #if i == 1
                    massDefct = 0.0
                    temp3 = i[1][0].split("-")
                    #temp = i[1][0].split("-")
                    #el1 = element(Z = int(temp[2]), N = int(temp[1]))
                    el1 = element(Z = int(temp3[2]), N = int(temp3[1]))
                    massDefct += df.loc[(df["Z"] == int(temp3[2])) & (df["N"] == int(temp3[1])), "MM"].values[0] * int(i[1][0][0])
                    mass1 = next((mass for e, mass in self.composition if el1.Z == el1.Z and e.N == el1.N), 0)
                    print(f"mass: {mass1}")
                    print(f"moles1: {self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1)}")
                    Y_1 = (self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1) * N_a)/self.volume

                    print(f"Y_1: {Y_1}")
                    #molCm3 = (0.5 * rate * ((self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1))/(self.volume)) ** 2)
                    molCm3 = (0.5 * Y_1 * Y_1 * rate * self.volume)/N_a
                    logging.info(molCm3)
                    print(f"molCM3: {molCm3}")
                    productEls = []
                    for w in i[2]:
                        _, N11, Z11 = w.split("-")
                        #                      \/The element                      \/The number of them produced
                        productEls.append((element(Z = int(Z11), N = int(N11)), w[0]))
                        massDefct += -df.loc[(df["Z"] == int(Z11)) & (df["N"] == int(N11)), "MM"].values[0]

                    totalChangeMoles = (molCm3) * SIMULATION_TIME_INTERVAL# * self.volume
                    for m, (element1, mass) in enumerate(self.composition):
                        if element1 == el1:
                            self.composition[m] = (el1, self.composition[m][1] - totalChangeMoles * self.returnMolarMass(el1))
                            print(f"mole change: {totalChangeMoles}")
                        if element1 in productEls:
                            self.composition[m] = (element, self.composition[m][1] + totalChangeMoles * self.returnMolarMass(element1))
                    for q in productEls:
                        self.composition.append((q[0], totalChangeMoles * self.returnMolarMass(q[0]) * int(q[1])))
                    self.calculateNumberOfMoles() #This might be too much, and may slow down the operation too much to justify the small amount of extra accuracy. If true, remove it.
                    deltaE = massDefct * mu * c * c
                    self.temperature += (2 * deltaE)/(3 * self.numberOfMoles * R)
                    print("RAN!")

    #Instead of calculating the r_0 and r_1 for all layers, instead just calculate r_1 i.e. the upper radius, and fix the lower radius of all layers to be the one below them.

    def calcAdibetHeat(self): #Untested
        oldPressure = self.pressure
        self.temperature = self.temperature*math.pow(((self.calculatePressure())/(oldPressure)), 0.4)

    def predictUpperBoundVelocity(self, forces: float): #Untested
        self.projectedVelocity = (forces*SIMULATION_TIME_INTERVAL)/self.mass + self.velocity

    def changeUpperBoundPosition(self): #Untested
        self.radius_1 += ((self.velocity + self.projectedVelocity) * SIMULATION_TIME_INTERVAL)/2
        self.velocity = self.projectedVelocity
        return self.radius_1

    def correctLowerBound(self, correctedValue: float): #Untested
        self.radius_0 = correctedValue

    def totalEnergy(self) -> dict["massEnergy": int, "kineticEnergy": int, "thermalEnergy": int]:
        return {"massEnergy": self.mass * c * c, "kineticEnergy": 0.5 * self.mass * self.velocity * self.velocity, "thermalEnergy": 1.5 * self.pressure * self.volume}

    def checkPresence(self, nuclei) -> bool:
        for nucleus in nuclei[1]:
            _, N, Z = nucleus.split("-")
            if (int(Z), int(N)) not in self.elementSet:
                return False
        return True

    def returnMolarMass(self, tup: element):
        total = df.loc[(df["Z"] == tup.Z) & (df["N"] == tup.N), "MM"].values[0]/1000
        return total

    def returnSpecificNumberOfMoles(self, elementt: element, mass: decimal.Decimal): # Works (tested C12, C13)
        #print(elementt.Z)
        #print(elementt.N)
        #moles = mass/df.loc[(df["Z"] == elementt.Z) & (df["N"] == elementt.N), "MM"].values[0] * 1000
        #return moles
        matches = df.loc[(df["Z"] == elementt.Z) & (df["N"] == elementt.N), "MM"]
        if matches.empty:
            raise ValueError(f"No molar mass found for Z={elementt.Z}, N={elementt.N}")
        return (mass * 1000)/ matches.values[0]



    def returnRadii(self): #Untested
        return (self.radius_0, self.radius_1)

    def changeRadii(self, newRadius0, newRadius1): #Untested
        self.radius_0 = newRadius0
        self.radius_1 = newRadius1

In [10]:
test = starLayer(composition=[(element(Z = 1, N = 1), 2*10**(29))], temperature=15000000, radius_0=0, radius_1=139000000, velocity=0)

testElement = element(Z = 6, N = 13)
e = test.returnSpecificNumberOfMoles(elementt = testElement, mass = 12.1)
logging.info(e)
pre = test.calculatePressure()
logging.info(pre)
test.calculateBaryonNumber()
test.calculateNumberOfMoles()


INFO:root:930.5290941364483
INFO:root:0.0


In [11]:
print(test.baryonNumber)

[2.39015328e+56]


In [12]:
test.fusion()
for i in test.composition:
    print(f"{i[0].Z}-{i[0].N}|{i[1]}")

INFO:root:8.104421070121509e-20
INFO:root:8.104710278365027e-20
INFO:root:141861385854318.06
INFO:root:[0.00055156]
INFO:root:[0.01118959]
INFO:root:[2.80020682e+13]
INFO:root:[227.55681065]
INFO:root:[0.1311185]
INFO:root:[2.2145337e-22]
INFO:root:[1.09391668e-07]


['p-p-d-ec']
volume: 1.1249494560988223e+25
rate: 8.104710278365027e-20
mass: 200000000000000000000000000000
moles1: 1.9844714474208457e+32
Y_1: 1.062338074460071e+31
molCM3: 141861385854318.06
mole change: 141861385854318.06
RAN!
['d-pg-he3-de04']
volume: 1.1249494560988223e+25
rate: [0.01118959]
moles1: 1.9844714474208453e+32, moles2: 141861385854318.06
Y_1: 1.062338074460071e+31
Y_2: 7594201049583.232
RAN!
['d-dn-he3-gi17']
volume: 1.1249494560988223e+25
rate: [227.55681065]
mass: [2.29324254e+11]
moles1: [1.13859318e+14]
Y_1: [6.09517907e+12]
molCM3: [0.1311185]
mole change: [0.1311185]
RAN!
['he3-p-he4-bet']
volume: 1.1249494560988223e+25
rate: [2.2145337e-22]
moles1: [2.80020682e+13], moles2: [1.98447145e+32]
Y_1: [1.49902198e+12]
Y_2: [1.06233807e+31]
RAN!
1-1|[2.e+29]
1-2|[2.29324254e+11]
2-3|[8.44550586e+10]
0-1|[0.00013225]
2-3|[0.00039546]
2-4|[4.37851448e-10]


In [13]:
print(10**(32)-10**(32)*e**(-16*10**(9)))

1e+32


In [14]:
class Star:
    def __init__(self, starLayers: [starLayer]): #Untested
        self.starLayers = starLayers

    def timeIncrement(self): #Untested
        #Todo
        #To make life simple, this is where all the time-dependant functions will be run
        self.doFusion(self)

    def doFusion(self): #Untested
        for i in self.starLayers:
            i.fusion(self)

    # The reason why the function determining the movement of the layers is not in the layers model is because the movement is dependent on the other layers as well.
    #\/ this function should only be run once the new velocity has been calculated.
    def move(self): #Untested
        prev = 0.0
        for i in self.starLayers:
            #todo
            i.correctLowerBound(self)
            prev = i.changeUpperBoundPosition()




# Nuclear Fusion and Quantum Tunneling

Nuclear fusion is the process by which smaller nuclei with low binding energies fuse into larger nuclei, with lager binding energies and release energy during this process. In order to fuse, nuclei need to touch, however the Coulomb force would mean that for the fusion of hydrogen, classical mechanics would suggest that a temperature on the order of $10^{10}$~$10^{11}$°K would be required, many orders of magnitude higher than the temperature at the core of the sun.

Quantum tunnelling is the effect where a particle which does not have enough energy to overcome a potential energy barrier e.g. an electric field or a solid object, but is still able to regardless. This small increase to the probability of overcoming the electric field between nuclei is what allows smaller stars which cannot undergo thermonuclear fusion to remain stable.

The rate at which fusion occurs is given by the equation:

$$r \simeq n_a n_b (\frac{8}{\pi m_R})^{1/2}\frac{S(E_0)}{(kT)^{3/2}}\frac{\Delta \sqrt{\pi}}{2}\exp{-\frac{3E_0}{kT}}$$

Where

$$\Delta = 4\sqrt{\frac{E_0 k T}{3}}$$
$$E_0 = \sqrt[3]{\frac{(\pi Z_a Z_b e^2)^2m_R(kT)^2}{2 \hbar^2}}$$

$S(E_0)$ is the S-Factor. However, since I cannot currently find a full table of the relevant S-Factors, I will instead use already calculated rates.

<small>The above equations were derived using equations found in sources 1 and 2<small>

# PP chain, CNO cycles and Helium capture
To model the actual fusion of the particles themselves, and to calculate what products they would form would be much too complicated. Because of this, I will predefine certain reactions that are allowed to happen in my star model.

I won't, however, bother to add the hot CNO cycles (for now), since they typically only occur duing supernovae, and that is far beyond the scope of this model.

## PP-I Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^3_2He \rightarrow ^4_2He + 2^1_1H$$

---
## PP-II Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^4_2He \rightarrow ^7_4be + \gamma$$

$$^7_4Be + ^0_0e^- \rightarrow ^7_3Li + v_e$$

$$^7_3Li + ^1_1H \rightarrow 2^4_2He$$


---
## PP-III Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^4_2He \rightarrow ^7_4He + \gamma$$

$$^7_4Be + ^1_1H \rightarrow ^8_5B + \gamma$$

$$^8_5B \rightarrow ^8_4Be + ^0_0e^+ + v_e$$

$$^8_4Be \rightarrow 2^4_2He$$

In the rates data, the last two reactions are listed as one, this is probably because its half life is ~0.77s.

---
## PP-IV Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^1_1H \rightarrow ^4_2He + ^0_0e^+ + v_e$$


---

## Helium capture

$$^4_2He + ^4_2He \rightarrow ^8_4Be$$
$$^4_2He + ^8_4Be\rightarrow ^{12}_6C$$
$$^4_2He + ^{12}_6C\rightarrow ^{16}_8O$$
$$^4_2He + ^{16}_8O\rightarrow ^{20}_10Ne$$
$$^4_2He + ^{20}_10Ne\rightarrow ^{24}_12Mg$$
$$...$$

This occurs all the way up to $^{56}_{28}Fe$, and then stops due to binding energy going down again.

---

### **CNO-I Cycle (Main Branch)**


$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$


---

### **CNO-II Cycle**


$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$

$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$


---

### **CNO-III Cycle**



$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$


---

### **CNO-IV Cycle**


$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$





# Displaying rseults!

I've decided to make a graphical display to show what's going on in the star.

In [15]:
#ChatGPT wrote this code to return an esitmate of colour in RGB for a black body of certain temperature

def blackbodyToRgb(tempK):
    temp = tempK / 100.0
    if temp <= 66:
        red = 255
        green = 99.4708025861 * math.log(temp) - 161.1195681661
        blue = 0 if temp <= 19 else 138.5177312231 * math.log(temp - 10) - 305.0447927307
    else:
        red = 329.698727446 * ((temp - 60) ** -0.1332047592)
        green = 288.1221695283 * ((temp - 60) ** -0.0755148492)
        blue = 255

    return tuple(max(0, min(255, int(c))) for c in (red, green, blue))

In [ ]:
pygame.init()

# Set up a fixed-size window
width, height = 1700, 1000
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Pygame Drawing Example")
clock = pygame.time.Clock()

font = pygame.font.SysFont("Arial", 16)
r = 10
text2 = font.render("0", True, (255, 255, 255))



running = True
while running:

    #WHERE ALL THE CALCULATIONS WILL HAPPEN

    #

    screen.fill((0, 0, 0))
    text1 = font.render(f"{r}", True, (255, 255, 255))
    pygame.draw.rect(screen, (255, 255, 255), (100, 50, 1500, 1))
    txtf = text2.get_rect()
    txtf.topright = (95, 42)

    pow = math.floor(math.log(r, 10))
    order = 10 ** pow
    t1 = 1500/r
    incr = t1 * pow
    num = math.floor(r / order)
    incr = t1 * order

    # Handle events (important to allow closing the window)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    startx = 100
    starty = 45
    for i in range(num+2):
        pygame.draw.rect(screen, (255, 255, 255), (startx + incr*(i-1), starty, 1, 10))
        temp = font.render(f"{i * order}", True, (255, 255, 255))
        temp2 = temp.get_rect()
        temp2.centerx = startx + incr*(i-1)
        screen.blit(temp, temp2)
    screen.blit(text1, (1605, 42))
    screen.blit(text2, txtf)



    pygame.display.flip()
    clock.tick(60)


pygame.quit()

# Tests

# Sources used:

The sources are *not* given in chronological order of access.

1. https://web.archive.org/web/20170115214447/https://zuserver2.star.ucl.ac.uk/~idh/PHAS2112/Lectures/Current/Part7.pdf

2. https://en.wikipedia.org/wiki/Stellar_nucleosynthesis#cite_note-40

3. https://www.youtube.com/@DrPhysicsA (All of nuclear playlist)

4. https://t2.lanl.gov/nis/data/astro/ for S-Factor experimental data

5. https://reaclib.jinaweb.org/popularRates.php for rates data



# How AI was used in this project

ChatGPT was used in the generation of the LaTEX displaying the CNO-I through to CNO-IV cycle.

ChatGPT was used to *double check* my equation for the rate of reaction between species A and specied B. The equation in the form that I give is the original that I derived using the given equations in sources 1 and 2.

Deepseek was used to write the code to parse data from Reaclib1 files.

ChatGPT was used in *no other aspect of this project*.